In [1]:
from symenergy.core import model
from sympy import init_printing
init_printing(pretty_print=False)
m = model.Model()
m.add_slot('t0', load=3000, vre=1)
m.add_plant('n', vc0=1, capacity=1000)
m.add_plant('g', vc0=2)
m.cache.delete()
m.generate_solve()



> 11:12:49 - WARNING - symenergy.core.model - !!! Monkey-patching sympy.linsolve !!!
> 11:12:50 - WARNING - symenergy.auxiliary.constrcomb - Aborting gen_col_combs: missing constraints ('p_cap_C',)
> 11:12:51 - WARNING - symenergy.core.model - Number of solutions with linear dependencies: Key 1: 0 (0.0%), Key 2: 0 (0.0%), Key 3: 0 (0.0%)


In [2]:
m.tc

g_p_t0*vc0_g_none*w_none + n_p_t0*vc0_n_none*w_none

In [19]:
import tabulate



In [4]:
from importlib import reload
from symenergy.evaluator import evaluator
from symenergy import logger
import numpy as np
reload(evaluator)

x_vals_vre = {m.slots['t0'].vre: np.linspace(0, 3000, 31)}
ev = evaluator.Evaluator(m, x_vals_vre, drop_non_optimum=False)

In [5]:
ev.cache_lambd.delete()
ev.get_evaluated_lambdas_parallel()

import inspect
print(inspect.getsource(
ev.df_lam_func.set_index(['idx', 'func']).loc[(1, 'g_p_t0'), 'lambd_func']
))

def _6f5e357ced970205c772e7346c30689d(vre_t0):
    return(3000 - vre_t0)



In [6]:
ev.cache_eval.delete()
ev.expand_to_x_vals_parallel()

ev.df_exp

> 11:13:00 - WARNING - symenergy.evaluator.evaluator - _call_eval: Generating dataframe with length 372


,func,idx,vre_t0,lambd,act_lb_n_pos_p_t0,act_lb_g_pos_p_t0,act_lb_n_p_cap_C_t0,is_positive,map_sanitize,mask_valid,is_optimum,slot,func_no_slot
0,g_p_t0,1,0.0,3000.0,True,False,False,1,False,True,False,t0,g_p
31,n_p_t0,1,0.0,0.0,True,False,False,1,False,True,False,t0,n_p
62,pi_supply_t0,1,0.0,2.0,True,False,False,0,False,True,False,t0,pi_supply
93,tc,1,0.0,6000.0,True,False,False,0,False,True,False,global,tc
124,g_p_t0,3,0.0,2000.0,False,False,True,1,False,True,True,t0,g_p
...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,tc,3,3000.0,-1000.0,False,False,True,0,False,False,False,global,tc
278,g_p_t0,4,3000.0,0.0,False,True,False,1,False,True,False,t0,g_p
309,n_p_t0,4,3000.0,0.0,False,True,False,1,False,True,False,t0,n_p
340,pi_supply_t0,4,3000.0,1.0,False,True,False,0,False,True,False,t0,pi_supply


In [7]:
(ev.df_bal.query('slot not in ["global"]')
   .pivot_table(index='vre_t0', columns='func',
                values='lambd')[['n_p_t0', 'g_p_t0', 'vre_t0', 'l_t0']].plot.area())

In [29]:
import tabulate
    tb = (ev.df_exp.query('is_optimum and func == "tc"')
            .groupby(['idx'] + m.constrs_cols_neq)
            .vre_t0.apply(lambda x: (x.min(), x.max()))
            .reset_index())
print(
tabulate.tabulate(tb, tb.columns, tablefmt='rst', showindex=False))

=====  ===================  =====================  ===================  ================
  idx  act_lb_n_pos_p_t0    act_lb_n_p_cap_C_t0    act_lb_g_pos_p_t0    vre_t0
=====  ===================  =====================  ===================  ================
    1  True                 False                  False                (3000.0, 3000.0)
    3  False                True                   False                (0.0, 2000.0)
    4  False                False                  True                 (2100.0, 2900.0)
=====  ===================  =====================  ===================  ================
